In [1]:
import os
import subprocess
import pandas as pd
import platform
import pathlib
import requests
from io import StringIO
import numpy as np
from annotations import *
from extract_video_features import *

In [2]:
filename_annotations ='https://docs.google.com/spreadsheets/d/1Rqu1sJiD-ogc4a6R491JTiaYacptOTqh6DKqhwTa8NA/gviz/tq?tqx=out:csv&sheet=Template'
features_folder = 'C:/Users/vince/Documents/TELECOM/Fil_Rouge/OpenFace/OpenFace_2.2.0/processed/'
video_folder = 'C:/Users/vince/Documents/TELECOM/Fil_Rouge/VideoAFPA'

In [3]:
video_names = [x.replace('.mp4','') for x in os.listdir(video_folder)]
video_names.remove('WIN_20210329_14_13_45_Pro')
video_names.remove('WIN_20210407_14_54_56_Pro_edit')
video_names.remove('WIN_20210403_18_49_15_Pro')
video_names.remove('WIN_20210407_15_02_22_Pro')

In [4]:
features_to_keep = [
#'frame','face_id','timestamp','confidence','success',
'AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r', 'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r', 'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r',
'AU01_c', 'AU02_c', 'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c', 'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c', 'AU28_c', 'AU45_c',
#'pose_Rx', 'pose_Ry', 'pose_Rz',
#'gaze_angle_x', 'gaze_angle_y',
'type_candidat','sexe',
'video_name','stress_global','stress','diapo'
]

In [15]:
list_df = []
for v_name in video_names:
    df_annoted = get_df_video_with_annotations(features_folder, v_name, filename_annotations)[features_to_keep]
    list_df.append(df_annoted)

In [16]:
df = pd.concat(list_df)
df = df.groupby(['video_name','stress_global','stress','diapo']).mean().reset_index()
df

,video_name,stress_global,stress,diapo,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,WIN_20210323_19_17_40_Pro,1,0,8,0.243693,0.003493,0.689740,0.006313,0.011067,0.119907,...,0.000000,0.478000,0.019333,0.526667,0.148000,0.184667,0.514000,0.708667,0.000000,0.342667
1,WIN_20210323_19_17_40_Pro,1,1,1,0.017888,0.005329,0.481358,0.056550,0.006495,0.026187,...,0.000000,0.721516,0.083962,0.051409,0.010917,0.124256,0.000000,0.007741,0.000000,0.304287
2,WIN_20210323_19_17_40_Pro,1,1,9,0.858888,0.275893,0.553935,0.096836,0.098443,0.164491,...,0.097035,0.314690,0.007075,0.436995,0.034704,0.287736,0.789757,0.878706,0.000000,0.378032
3,WIN_20210323_19_17_40_Pro,1,1,10,1.886323,1.042617,0.431980,0.121930,0.130602,0.129493,...,0.105473,0.322886,0.036816,0.385075,0.033333,0.311443,0.847264,0.872637,0.000000,0.379602
4,WIN_20210323_19_17_40_Pro,1,1,11,1.946727,1.109892,0.317241,0.274298,0.053301,0.079083,...,0.018763,0.313412,0.097637,0.301251,0.059416,0.285962,0.668867,0.765810,0.005559,0.259903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,WIN_20210417_14_53_12_Pro,0,0,10,0.305747,0.195752,0.218913,0.037692,0.222918,0.720764,...,0.002978,0.019355,0.173201,0.174690,0.129032,0.933499,0.145409,0.132506,0.000000,0.159305
180,WIN_20210417_14_53_12_Pro,0,0,11,0.203982,0.097375,0.292386,0.051499,0.321697,0.989041,...,0.001731,0.003809,0.052632,0.049169,0.092452,0.930055,0.131579,0.074792,0.003809,0.168629
181,WIN_20210417_14_53_12_Pro,0,1,12,0.356883,0.061600,0.395706,0.011831,0.277116,0.988301,...,0.002064,0.018167,0.000000,0.028902,0.218002,0.741949,0.183732,0.107969,0.001032,0.124690
182,WIN_20210417_14_53_12_Pro,0,1,17,0.104461,0.011571,0.638750,0.054232,0.365562,1.489920,...,0.000000,0.067661,0.108945,0.329128,0.014908,0.994266,0.266055,0.331422,0.049312,0.232798


In [18]:
x = df.drop(['video_name','stress_global','stress','diapo'],axis=1)
y = df[['stress']]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
error = np.mean(np.abs(y_pred - y_test))
print(error)

0.625


In [ ]:
#extract_video_features('C:/Users/vince/Documents/TELECOM/Fil_Rouge/OpenFace/OpenFace_2.2.0/',
#'C:/Users/vince/Documents/TELECOM/Fil_Rouge/VideoAFPA/WIN_20210407_15_02_22_Pro.mp4')